In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

dataset = pd.read_csv('https://object.cato.org/sites/cato.org/files/human-freedom-index-files/human-freedom-index-2019.csv')

# Nos quedamos sólo con las que comienzan con pf_indentity, junto con otras columnas más generales
score_cols = [col for col in dataset.columns if 'pf_identity' in col] + [
    'pf_score', # Personal Freedom (score)
    'pf_rank', # Personal Freedom (rank)
    'ef_score', # Economic Freedom (score)
    'ef_rank', # Economic Freedom (rank)
    'hf_score', # Human Freedom (score)
    'hf_rank', # Human Freedom (rank)
]

important_cols = ['year', 'ISO_code', 'countries', 'region'] + score_cols

# Casi todas las columnas se leen como objetos debido a que los valores nulos están marcados con -. Los vamos a eliminar y volveremos a convertir las columnas con puntajes a tipo numérico.
dataset = dataset[important_cols].replace('-', np.nan)
for score_col in score_cols:
    dataset[score_col] = pd.to_numeric(dataset[score_col])
dataset

# Por ahora las filas con NaN de hf_score y pf_identity las eliminaremos.
dataset = dataset.dropna(subset=['hf_score', 'pf_identity'])

dataset

,year,ISO_code,countries,region,pf_identity_legal,pf_identity_sex_male,pf_identity_sex_female,pf_identity_sex,pf_identity_divorce,pf_identity,pf_score,pf_rank,ef_score,ef_rank,hf_score,hf_rank
0,2017,ALB,Albania,Eastern Europe,0.0,10.0,10.0,10.0,7.5,5.8,8.01,46.0,7.67,30.0,7.84,38.0
1,2017,DZA,Algeria,Middle East & North Africa,NaN,0.0,0.0,0.0,0.0,0.0,5.20,146.0,4.77,159.0,4.99,155.0
2,2017,AGO,Angola,Sub-Saharan Africa,10.0,0.0,0.0,0.0,5.0,5.0,5.98,121.0,4.83,158.0,5.40,151.0
3,2017,ARG,Argentina,Latin America & the Caribbean,10.0,10.0,10.0,10.0,10.0,10.0,8.04,41.0,5.67,147.0,6.86,77.0
4,2017,ARM,Armenia,Caucasus & Central Asia,7.0,10.0,10.0,10.0,7.5,8.2,7.15,72.0,7.70,27.0,7.42,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,2008,AUS,Australia,Oceania,NaN,10.0,10.0,10.0,10.0,10.0,9.29,7.0,8.18,6.0,8.73,4.0
1616,2008,DNK,Denmark,Western Europe,NaN,10.0,10.0,10.0,10.0,10.0,9.49,3.0,7.98,9.0,8.73,4.0
1617,2008,CHE,Switzerland,Western Europe,NaN,10.0,10.0,10.0,10.0,10.0,9.31,6.0,8.35,4.0,8.83,3.0
1618,2008,NZL,New Zealand,Oceania,NaN,10.0,10.0,10.0,10.0,10.0,9.42,4.0,8.46,3.0,8.94,2.0


# Parte 1

## 1. Estadísticos descriptivos

  1. Para comenzar con un pantallazo de los datos, calcular el rango de las variables.
  3. Para cada país, tenemos varias realizaciones para cada variable *pf_identity* y *hf_score*. Si queremos comparar un país con otro, ¿cuál es la manera adecuada de hacerlo? Por ejemplo, ¿nos quedamos con un único valor? ¿o comparamos todos los valores? ¿usamos el promedio? ¿usamos la mediana? 
  2. Obtener media, mediana y desviación estándar de las variables *pf_identity* y *hf_score* en el mundo y compararla con la de Latinoamérica y el caribe. Usar la respuesta del punto anterior para justificar si la comparación es válida o no.
  3. ¿Tiene sentido calcular la moda?
  4. ¿Cómo pueden sanearse los valores faltantes?
  5. ¿Encuentra outliers en estas dos variables? ¿Qué método utiliza para detectarlos? Los outliers, ¿son globales o por grupo? ¿Los eliminaría del conjunto de datos?
  

### 1.1 Rango de las variables

In [22]:
# 1.1 Rango de las variables

ranges = pd.concat([dataset.min(), dataset.max()], axis=1) 
ranges.columns = ['Minimo', 'Maximo']
print("Rangos de las variables: \n", ranges)

Rangos de las variables: 
                                          Minimo          Maximo
year                                       2008            2017
ISO_code                                    AGO             ZWE
countries                               Albania        Zimbabwe
region                  Caucasus & Central Asia  Western Europe
pf_identity_legal                             0              10
pf_identity_sex_male                          0              10
pf_identity_sex_female                        0              10
pf_identity_sex                               0              10
pf_identity_divorce                           0              10
pf_identity                                   0              10
pf_score                                   2.22            9.56
pf_rank                                       1             162
ef_score                                   2.58            9.11
ef_rank                                       1             162
hf_score     

### 1.2 Para este punto, podemos tomar a Argentina y Armenia para comparar los valores pf_identity y hf_score. Procedemos a calcular el promedio de la mitad de las agregaciones por pais.

In [32]:
# 1.2 Comparación de hf_score y pf_identity de Argentina con Armenia.

# Procedemos a calcular el dataset argentino para las dos variables y tomaremos solo su mitad para compararlo con Armenia
argentina_dataset = dataset.loc[dataset['countries']=='Argentina',['year','pf_identity','hf_score']]
argentina_dataset.sort_values(by=['year']) # Ordenamos por columna año
argentina_dataset_to_compare = argentina_dataset.head(int(len(argentina_dataset)/2)) # Dividimos el tamaño del dataset argentino en 2
print(argentina_dataset_to_compare)

# Procedemos a calcular el dataset de Armenia para las dos variables y tomaremos solo su mitad para compararlo con Argentina
armenia_dataset = dataset.loc[dataset['countries']=='Armenia',['year','pf_identity','hf_score']]
armenia_dataset.sort_values(by=['year']) # Ordenamos por columna año
armenia_dataset_to_compare = armenia_dataset.head(int(len(armenia_dataset)/2)) # Dividimos el tamaño del dataset de Armenia en 2
print(armenia_dataset_to_compare)

hf_score_argentina_mean = argentina_dataset_to_compare['hf_score'].mean()
hf_score_armenia_mean = armenia_dataset_to_compare['hf_score'].mean()

pf_identity_argentina_mean = argentina_dataset_to_compare['pf_identity'].mean()
pf_identity_armeina_mean = armenia_dataset_to_compare['pf_identity'].mean()

print("Media de hf_score de Argentina: %f, hf_score de Armenia: %f" %(hf_score_argentina_mean, hf_score_armenia_mean))
print("Media de pf_identity de Argentina: %f, pf_identity de Armenia: %f" %(pf_identity_argentina_mean, pf_identity_armeina_mean))



     year  pf_identity  hf_score
3    2017         10.0      6.86
165  2016         10.0      6.75
327  2015         10.0      6.54
489  2014         10.0      6.27
651  2013         10.0      6.55
     year  pf_identity  hf_score
4    2017          8.2      7.42
166  2016          8.2      7.44
328  2015          8.2      7.34
490  2014          8.8      7.43
652  2013          8.8      7.53
Media de hf_score de Argentina: 6.594000, hf_score de Armenia: 7.432000
Media de pf_identity de Argentina: 10.000000, pf_identity de Armenia: 8.440000


Creemos que tomar los ultimos 5 años de medicione por cada pais, nos puede dar un índice significativo de la situacion actual de libertad para cada variable. Consideramos que la media es la medicion correcta para este caso. La mediana y la moda no creo que serían significativas para comparar las variables entre paises.

### 1.3 Media, mediana y desviación estandar para pf_identity y hf_score


In [9]:
# 1.3 Media, mediana y desviación estandar para pf_identity y hf_score

hf_score_mean_world = dataset['hf_score'].mean()
pf_identity_mean_world = dataset['pf_identity'].mean()

hf_score_median_world = dataset['hf_score'].median()
pf_identity_median_world = dataset['pf_identity'].median()

hf_score_std_world = dataset['hf_score'].std()
pf_identity_std_world = dataset['pf_identity'].std()

hf_score_mean_latinamerica_caribbean = dataset[(dataset.region == "Latin America & the Caribbean")]['hf_score'].mean()
pf_identity_mean_latinamerica_caribbean = dataset[(dataset.region == "Latin America & the Caribbean")]['pf_identity'].mean()

hf_score_median_latinamerica_caribbean = dataset[(dataset.region == "Latin America & the Caribbean")]['hf_score'].median()
pf_identity_median_latinamerica_caribbean = dataset[(dataset.region == "Latin America & the Caribbean")]['pf_identity'].median()


hf_score_std_latinamerica_caribbean = dataset[(dataset.region == "Latin America & the Caribbean")]['hf_score'].std()
pf_identity_std_latinamerica_caribbean = dataset[(dataset.region == "Latin America & the Caribbean")]['pf_identity'].std()

print("Media de hf_score del mundo: %f, hf_score de latinoamerica y el caribe: %f" %(hf_score_mean_world, hf_score_mean_latinamerica_caribbean))
print("Mediana de hf_score del mundo: %f, hf_score de latinoamerica y el caribe: %f" %(hf_score_median_world, hf_score_median_latinamerica_caribbean))
print("Desviación estandar de hf_score del mundo: %f, hf_score de latinoamerica y el caribe: %f" %(hf_score_std_world, hf_score_std_latinamerica_caribbean))

print("-----------------")

print("Media de pf_identity del mundo: %f, pf_identity de latinoamerica y el caribe: %f" %(pf_identity_mean_world, pf_identity_mean_latinamerica_caribbean))
print("Mediana de pf_identity del mundo: %f, pf_identity de latinoamerica y el caribe: %f" %(pf_identity_median_world, pf_identity_median_latinamerica_caribbean))
print("Desviación estandar de pf_identity del mundo: %f, pf_identity de latinoamerica y el caribe: %f" %(pf_identity_std_world, pf_identity_std_latinamerica_caribbean))


Media de hf_score del mundo: 6.978214, hf_score de latinoamerica y el caribe: 7.036977
Mediana de hf_score del mundo: 6.900000, hf_score de latinoamerica y el caribe: 7.020000
Desviación estandar de hf_score del mundo: 1.056239, hf_score de latinoamerica y el caribe: 0.634897
-----------------
Media de pf_identity del mundo: 7.280649, pf_identity de latinoamerica y el caribe: 8.582171
Mediana de pf_identity del mundo: 8.800000, pf_identity de latinoamerica y el caribe: 10.000000
Desviación estandar de pf_identity del mundo: 3.181577, pf_identity de latinoamerica y el caribe: 1.875832
